# Financial Analysis with Data Science
By Cyrill Adrian Wicaksono

This note book aims to extracting the crucial insights from financial markets.

# Essential Libraries



1.   **yfinance**: extract data from the financial market
2.  **Quanstats**: quantitative financial analysis tools
3. **PyPortofolioOpt**: optimization and asset allocation
4. **TA**: technical analysis library; provides multiple technical analysis indicators



In [30]:
#Installing yfinance
!pip install yfinance

#Installing Quantstats
!pip install quantstats

#Installing PyPortfolioOpt
!pip install pyportfolioopt

#Installing TA
!pip install ta

In [31]:
#Import libraries
import pandas as pd
from pandas_datareader import data
import numpy as np
from scipy import stats

import quantstats as qs
import ta
import yfinance as yf

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

# Fundamental Analysis
This is the exploration process; using stocks for 4 tickers namely: BBRI, BBCA, BMRI, and BBNI


In [32]:
tickers = ["BBCA.JK", "BBNI.JK", "BMRI.JK", "BBRI.JK"]

## Average Daily Returns
A stock daily return is the percentual change in price over a single day. We calculate it between the difference between the stock's closing price on one day and its closing price the day before, dividing the result by closing of the day before, and multiplying it by 100.


For example:
<center>$((price On Monday - priceOnTuesday) \div 100) \times 100 = Daily Returns$</center>

In [33]:
returns_data = {}

for ticker in tickers:
    returns = qs.utils.download_returns(ticker)
    returns_data[ticker] = returns.loc['2010-07-01': '2024-02-10']


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed



Calculate the average daily returns of each stock from the data


In [34]:

for ticker in tickers:
  print(f"Average Daily Returns {ticker} : {returns_data[ticker].mean()}")

Average Daily Returns BBCA.JK : 0.000746378538102235
Average Daily Returns BBNI.JK : 0.0007087290967559536
Average Daily Returns BMRI.JK : 0.0006728326821862299
Average Daily Returns BBRI.JK : 0.0007871482361318915


## Beta and Alpha
Beta and Alpha are the two key matrix used in finance to evaluate the performance of a stock relative to the overall market. Beta is a measure of a stock vollatility compared to the market. Alpha, on the other hand, is a measurement of a stock excess return relative to its expected performance based on its Beta.

In [35]:
ihsg = qs.utils.download_returns("^JKSE")
ihsg = ihsg.loc['2010-07-01': '2024-02-10']

returns_data["IHSG.JK"] = ihsg

[*********************100%%**********************]  1 of 1 completed


In [36]:
returns_no_index = {}

for ticker in tickers:
  returns_no_index[ticker] = returns_data[ticker].reset_index(drop=True)

returns_no_index['IHSG.JK'] = returns_data['IHSG.JK'].reset_index(drop=True)

In [37]:
def compute_alpha_beta(stock):
  result= {}
  X = returns_no_index["IHSG.JK"].values.reshape(-1, 1)
  y = stock.values.reshape(-1, 1)


  if len(X) != len(y):
      min_len = min(len(X), len(y))
      X = X[:min_len]
      y = y[:min_len]

  model = LinearRegression()
  model.fit(X, y)

  beta = model.coef_[0]
  alpha = model.intercept_

  result['beta'] = beta.round(3)
  result['alpha'] = alpha.round(3)
  return result




In [38]:
for stock in returns_no_index.keys():
  result = compute_alpha_beta(returns_no_index[stock])
  print(f"{stock}: {result}")


BBCA.JK: {'beta': array([0.411]), 'alpha': array([0.001])}
BBNI.JK: {'beta': array([0.473]), 'alpha': array([0.001])}
BMRI.JK: {'beta': array([0.535]), 'alpha': array([0.])}
BBRI.JK: {'beta': array([0.532]), 'alpha': array([0.001])}
IHSG.JK: {'beta': array([1.]), 'alpha': array([0.])}


Beta values for all the stocks are greater than 1, meaning that they are more volatile than the benchmark and may offer higher returns, but also come with greater risks. On the other hand, the alpha values for all the stocks are small, close to zero, suggesting that there is little difference between the expected returns and the risk-adjusted returns

## Sharpe Ratio

Sharpe ratio measures of risk-adjusted return of investments. It is calculated by dividing the average excess return of investments over the standard deviation of returns. Here is the equation:

<div style="display: flex; justify-content: center;">
  <center><img src="https://latex.codecogs.com/png.image?\dpi{150}&space;\bg_white&space;\text{Sharpe&space;ratio}&space;=&space;\frac{R_p&space;-&space;R_f}{\sigma_p}" title="\text{Sharpe ratio} = \frac{R_p - R_f}{\sigma_p}" /></center>
</div>

where <b><i>R<sub>p</sub></i></b> is the average return of the investment, <i><b>R<sub>f</sub></b></i> is the risk-free rate of return, and <i><b>σ<sub>p</sub></b></i> is the standard deviation of the returns. The average excess return is the difference between the average return of the investment and the risk-free rate of return, typically represented by a government bond. The standard deviation is a measurement of the volatility of returns.</p>
<br>

A higher Sharpe ratio indicates that an investment provides higher returns for a given level of risk compared to other investments with a lower Sharpe ratio. In general, a Sharpe ratio greater than 1 is considered good, while a Sharpe ratio less than 1 is considered poor. A Sharpe ratio of 1 means that the investment's average return is equal to the risk-free rate of return.

In general, a Sharpe ratio under 1.0 is considered bad, equal to 1.0 is considered acceptable or good, 2.0 or higher is rated as very good, and 3.0 or higher is considered excellent.


In [39]:
for stock in returns_data.keys():
  print(f"Sharpe Ratio for {stock}: {qs.stats.sharpe(returns_data[stock]).round(2)}")

Sharpe Ratio for BBCA.JK: 0.75
Sharpe Ratio for BBNI.JK: 0.53
Sharpe Ratio for BMRI.JK: 0.52
Sharpe Ratio for BBRI.JK: 0.6
Sharpe Ratio for IHSG.JK: 0.51


None of the stocks have a Sharp ratio above 1, which may suggest that these investments' average returns are beneath the risk-free rate of return.

It's important to note that the Sharpe ratio is an annual metric and, since the beginning of 2022, the market, in general, has been bearish, with prices going down over the past year.

## Piotroski F-Score

The Piotroski F-score is a scoring system designed by Joseph Piotroski to assess the financial strength of a company based on its fundamental characteristics. The score ranges from 0 to 9, with a higher score indicating better financial health. Here's how the F-score is calculated based on the provided code:

1. **Net Income (NI) > 0 (Criteria 1):**
   - If the net income of the latest fiscal year is greater than zero, assign 1 point.

2. **Operating Cash Flow (OCF) > 0 (Criteria 2):**
   - If the operating cash flow of the latest fiscal year is greater than zero, assign 1 point.

3. **Return on Assets (ROA) > 0 (Criteria 3):**
   - Calculate the return on assets (net income divided by total assets).
   - If the ROA is greater than zero, assign 1 point.

4. **Cash Flow from Operations (OCF) > Net Income (NI) (Criteria 4):**
   - If the cash flow from operations of the latest fiscal year is greater than the net income, assign 1 point.

5. **Long-Term Debt (LT Debt) Decreased (Criteria 5):**
   - If the long-term debt decreased from the previous fiscal year to the latest fiscal year, assign 1 point.

6. **Current Ratio Increased (Criteria 6):**
   - If the current ratio (total current assets divided by total current liabilities) increased from the previous fiscal year to the latest fiscal year, assign 1 point.

7. **Shares Outstanding (SO) Decreased (Criteria 7):**
   - If the number of shares outstanding decreased from the previous fiscal year to the latest fiscal year, assign 1 point.

8. **Gross Margin (GM) Increased (Criteria 8):**
   - If the gross margin (total revenue minus cost of revenue) increased from the previous fiscal year to the latest fiscal year, assign 1 point.

Finally, the total Piotroski F-score is the sum of points assigned based on meeting each of the criteria.

In [40]:
def compute_piotroski_score(ticker):
    # Fetch balance sheet data
    balance_sheet = yf.Ticker(ticker).balance_sheet

    # Fetch cash flow data
    cash_flow = yf.Ticker(ticker).cashflow

    # Fetch financial statement data
    financials = yf.Ticker(ticker).financials

    # Concatenate the data vertically
    data = pd.concat([balance_sheet, cash_flow, financials]).transpose()

    result = {}

    # Criteria 1: Net Income (NI) > 0
    try:
        if data['Net Income'][-1] > 0:
            result["net_income"] = data['Net Income'][-1]
    except (KeyError, IndexError, TypeError) as e:
        print(f"Error in Criteria 1: {e}")

    # Criteria 2: Operating Cash Flow (OCF) > 0
    try:
        if data['Operating Cash Flow'][-1] > 0:
            result["operating_cashflow"] = data['Operating Cash Flow'][-1]
    except (KeyError, IndexError, TypeError) as e:
        print(f"Error in Criteria 2: {e}")

    # Criteria 3: Return on Assets (ROA) > 0
    try:
        if data['Total Assets'][-1] > 0:
            roa = data['Net Income'][-1] / data['Total Assets'][-1]
            if roa > 0:
                result['roa'] = roa
    except (KeyError, IndexError, TypeError, ZeroDivisionError) as e:
        print(f"Error in Criteria 3: {e}")

    # Criteria 4: Cash Flow from Operations (OCF) > Net Income (NI)
    try:
        if data['Operating Revenue'][-1] > data['Net Income'][-1]:
            result['op_revenue_v_net_income_margin'] = data['Operating Revenue'][-1] - data['Net Income'][-1]
    except (KeyError, IndexError, TypeError) as e:
        print(f"Error in Criteria 4: {e}")

    # Criteria 5: Long-Term Debt (LT Debt) Decreased
    try:
        if data['Long Term Debt And Capital Lease Obligation'][-1] < data['Long Term Debt And Capital Lease Obligation'][-2]:
                result['lt_debt_decreased_percent'] = (data['Long Term Debt And Capital Lease Obligation'][-2] - data['Long Term Debt And Capital Lease Obligation'][-1]) / data['Long Term Debt And Capital Lease Obligation'][-2] * 100
    except (KeyError, IndexError, TypeError, ZeroDivisionError) as e:
        print(f"Error in Criteria 5: {e}")

    # Criteria 6: Current Ratio Increased
    try:
        if data['Total Liabilities Net Minority Interest' ][-1] > 0 and data['Total Assets'][-1] > data['Total Liabilities Net Minority Interest' ][-1]:
                result['current_ratio'] = data['Total Liabilities Net Minority Interest' ][-1] / data['Total Assets'][-1]
    except (KeyError, IndexError, TypeError, ZeroDivisionError) as e:
        print(f"Error in Criteria 6: {e}")

    # Criteria 7: Shares Outstanding (SO) Decreased
    try:
        if data['Diluted Average Shares'][-1] < data['Diluted Average Shares'][-2]:
                result['percent_shares_outstanding'] = (data['Diluted Average Shares'][-2] - data['Diluted Average Shares'][-1]) / data['Diluted Average Shares'][-2] * 100
    except (KeyError, IndexError, TypeError, ZeroDivisionError) as e:
        print(f"Error in Criteria 7: {e}")

    # Criteria 8: Gross Margin (GM) Increased
    try:
        if data['Total Revenue'][-1] > data['Operating Revenue'][-1]:
                result['gross_margin'] = data['Total Revenue'][-1] - data['Operating Revenue'][-1]
    except (KeyError, IndexError, TypeError) as e:
        print(f"Error in Criteria 8: {e}")

    return result



In [41]:
for ticker in tickers:
  print(f"Piotroski F-Score for {ticker}: {compute_piotroski_score(ticker)}")

Piotroski F-Score for BBCA.JK: {'net_income': 27131109000000.0, 'operating_cashflow': 50978875000000.0, 'roa': 0.025224859881212633, 'op_revenue_v_net_income_margin': 49575335000000.0, 'current_ratio': 0.8282634649205101}
Piotroski F-Score for BBNI.JK: {'net_income': 3280403000000.0, 'operating_cashflow': 74253924000000.0, 'roa': 0.0036803155662402487, 'op_revenue_v_net_income_margin': 51066081000000.0, 'lt_debt_decreased_percent': 11.411306064828983, 'current_ratio': 0.8733675981349038}
Piotroski F-Score for BMRI.JK: {'net_income': 16799515000000.0, 'operating_cashflow': 109894642000000.0, 'roa': 0.010894877456675047, 'op_revenue_v_net_income_margin': 70293022000000.0, 'lt_debt_decreased_percent': 5.184078212604047, 'current_ratio': 0.8672474890922705}
Piotroski F-Score for BBRI.JK: {'net_income': 18654753000000.0, 'operating_cashflow': 29804584000000.0, 'roa': 0.011586332858798556, 'op_revenue_v_net_income_margin': 105487029000000.0, 'current_ratio': 0.8574797707092315, 'percent_shar

## Revenue Growth

Calculate the percentage change in revenue over a specified period (e.g., year-over-year growth rate).

In [42]:
def compute_revenue_growth(ticker):
    revenue_growth = 0
    # Fetch historical revenue data
    revenue_data = yf.Ticker(ticker).financials.transpose()['Total Revenue']

    # Compute revenue growth percentage
    current_revenue = revenue_data[-1]
    previous_revenue = revenue_data[-2]
    revenue_growth = ((current_revenue - previous_revenue) / previous_revenue) * 100


    return revenue_growth

In [43]:
for ticker in tickers:
  growth_percentage = compute_revenue_growth(ticker)
  print(f"Revenue Growth {ticker}: {growth_percentage:.2f}%")

Revenue Growth BBCA.JK: -4.28%
Revenue Growth BBNI.JK: -4.78%
Revenue Growth BMRI.JK: -11.96%
Revenue Growth BBRI.JK: -16.01%


## Earning Per Share

Divide the company's net income by the number of outstanding shares.

In [44]:
def compute_eps(ticker):
  eps = 0
  net_income = yf.Ticker(ticker).financials.transpose()['Net Income']
  outstanding_shares = yf.Ticker(ticker).info['sharesOutstanding']

  eps = net_income[-1]/outstanding_shares

  return eps

In [45]:
for ticker in tickers:
  print(f"EPS scores for {ticker}: {compute_eps(ticker)}")

EPS scores for BBCA.JK: 220.0860563092296
EPS scores for BBNI.JK: 5668.729079429653
EPS scores for BMRI.JK: 179.99486939812365
EPS scores for BBRI.JK: 123.71264530913241


## Price-to-Earning

Divide the current market price per share by the earnings per share.

In [46]:
def compute_pe_ratio(ticker):
  pe_ratio = 0

  try:
    current_price = yf.Ticker(ticker).info['currentPrice']

    eps = yf.Ticker(ticker).financials.transpose()['Net Income'][-1] / yf.Ticker(ticker).info['sharesOutstanding']
    pe_ratio = current_price/eps
  except KeyError as e:
    print(f"KeyError: {e}")

  return pe_ratio

In [47]:
for ticker in tickers:
  print(f"PE Ratio for {ticker}: {compute_pe_ratio(ticker)}")

PE Ratio for BBCA.JK: 46.345507621203396
PE Ratio for BBNI.JK: 1.0893094225313171
PE Ratio for BMRI.JK: 40.8344972530457
PE Ratio for BBRI.JK: 49.50989435796872


## Debt-to-Equity

Divide total debt by shareholders' equity.

In [48]:
def compute_debt_to_equity_ratio(ticker):
  debt_to_equity_ratio = 0
  try:
      total_stockholder_equity = yf.Ticker(ticker).balance_sheet.transpose()['Stockholders Equity']
      total_debt = yf.Ticker(ticker).info['totalDebt']
      debt_to_equity_ratio = total_debt / total_stockholder_equity[-1]

  except KeyError as e:
    print(f"KeyError: {e}")

  return debt_to_equity_ratio

In [49]:
for ticker in tickers:
  print(f"Debt-to-Equity for {ticker}: {compute_debt_to_equity_ratio(ticker)}")

Debt-to-Equity for BBCA.JK: 0.03749632728183334
Debt-to-Equity for BBNI.JK: 0.48998715045413244
Debt-to-Equity for BMRI.JK: 1.018590890563945
Debt-to-Equity for BBRI.JK: 0.7651531417484786


## Free Cash Flow

Subtract capital expenditures from operating cash flow.

In [50]:
def compute_free_cashflow(ticker):
  financial = yf.Ticker(ticker).cashflow.transpose()

  op_cf = financial['Operating Cash Flow'][-1]
  capex = financial['Capital Expenditure'][-1]

  return  op_cf - capex


In [51]:
for ticker in tickers:
  print(f"Free Cashflow of {ticker}: {compute_free_cashflow(ticker)}")

Free Cashflow of BBCA.JK: 53652612000000.0
Free Cashflow of BBNI.JK: 75527364000000.0
Free Cashflow of BMRI.JK: 116599094000000.0
Free Cashflow of BBRI.JK: 34179650000000.0


## Return On Equity

Divide net income by shareholders' equity.

In [52]:
def compute_return_on_equity(ticker):
  total_stockholder_equity = yf.Ticker(ticker).balance_sheet.transpose()['Stockholders Equity'][-1]
  net_income = yf.Ticker(ticker).financials.transpose()['Net Income'][-1]

  roe = (net_income/total_stockholder_equity) * 100
  return roe


In [53]:
for ticker in tickers:
  print(f"Free Return On Equity of {ticker}: {compute_return_on_equity(ticker)}")

Free Return On Equity of BBCA.JK: 14.697534662742962
Free Return On Equity of BBNI.JK: 2.976974299162956
Free Return On Equity of BMRI.JK: 8.870868759396
Free Return On Equity of BBRI.JK: 8.22099314605118


## Dividend Yield

Divide the annual dividend per share by the current market price per share.

In [54]:
def compute_dividend_yield(ticker):
  dividends = yf.Ticker(ticker).dividends
  current_price = yf.Ticker(ticker).history(period='1d')['Close'][0]

  annual_dividend_per_share = dividends.sum()
  dividend_yield = (annual_dividend_per_share/current_price) * 100

  return dividend_yield


In [55]:
for ticker in tickers:
  print(f"Dividend Yield of {ticker}: {compute_dividend_yield(ticker)}")

Dividend Yield of BBCA.JK: 10.258190709046456
Dividend Yield of BBNI.JK: 25.69776464777328
Dividend Yield of BMRI.JK: 20.205575265306123
Dividend Yield of BBRI.JK: 23.223468930612245


# Pipelining

## Get Tickers


Get all the ticker that is publicly traded from the Jakarta Stock Exchange.

In [56]:
ticker_list = pd.read_csv("/content/ticker_symbols.csv")

tickers = list(ticker_list['Ticker'])

In [57]:
len(tickers)

921

## Computation Function of the Financial Analysis

In [58]:
stock_analysis = {}

In [59]:
def compute_financial_analysis(ticker):
    result = {}

    try:
        # Compute Average Daily Returns
        returns = qs.utils.download_returns(ticker).loc['2020-07-01': '2024-02-10']
        result['Average Daily Returns'] = returns.mean()
    except Exception as e:
        result['Average Daily Returns'] = ""

    try:
        # Compute Beta-Alpha
        returns_no_index = returns.reset_index(drop=True)
        result['Beta Alpha'] = compute_alpha_beta(returns_no_index)
    except Exception as e:
        result['Beta Alpha'] = ""

    try:
        # Compute Sharpe Ratio
        result['Sharpe Ratio'] = qs.stats.sharpe(returns).round(2)
    except Exception as e:
        result['Sharpe Ratio'] = ""

    try:
        # Compute Piotroski F-score
        result['Piotroski F-Score'] = compute_piotroski_score(ticker)
    except Exception as e:
        result['Piotroski F-Score'] = ""

    try:
        # Compute Revenue Growth
        result['Revenue Growth'] = compute_revenue_growth(ticker)
    except Exception as e:
        result['Revenue Growth'] = ""

    try:
        # Compute Earning Per Share
        result['Earning per Share'] = compute_eps(ticker)
    except Exception as e:
        result['Earning per Share'] = ""

    try:
        # Compute Price to Earning
        result['Price to Earning'] = compute_pe_ratio(ticker)
    except Exception as e:
        result['Price to Earning'] = ""

    try:
        # Compute Debt to Equity
        result['Debt to Equity'] = compute_debt_to_equity_ratio(ticker)
    except Exception as e:
        result['Debt to Equity'] = ""

    try:
        # Compute Free Cashflow
        result['Free Cash Flow'] = compute_free_cashflow(ticker)
    except Exception as e:
        result['Free Cash Flow'] = ""

    try:
        # Compute Return on Equity
        result['Return on Equity'] = compute_return_on_equity(ticker)
    except Exception as e:
        result['Return on Equity'] = ""

    try:
        # Compute Dividend Yield
        result['Dividend Yield'] = compute_dividend_yield(ticker)
    except Exception as e:
        result['Dividend Yield'] = ""

    return result


Check for one of the ticker

In [60]:
test_ticker = tickers[0]
test_ticker

'AALI.JK'

In [61]:
print(f"{test_ticker}: {compute_financial_analysis(test_ticker)}")

[*********************100%%**********************]  1 of 1 completed


Error in Criteria 2: 'Operating Cash Flow'
AALI.JK: {'Average Daily Returns': -2.992250227229979e-05, 'Beta Alpha': {'beta': array([0.035]), 'alpha': array([-0.])}, 'Sharpe Ratio': -0.03, 'Piotroski F-Score': {'net_income': 833090000000.0, 'roa': 0.0299875120724492, 'op_revenue_v_net_income_margin': 17973953000000.0, 'current_ratio': 0.30716554640793275}, 'Revenue Growth': -22.67492030276398, 'Earning per Share': 432.8437198839821, 'Price to Earning': 15.47902786205572, 'Debt to Equity': 0.2135743776992211, 'Free Cash Flow': '', 'Return on Equity': 4.442555984422978, 'Dividend Yield': 133.69811938805972}


### Computes the Analysis for All the Tickers

In [62]:
for ticker in tickers:
  stock_analysis[ticker] = compute_financial_analysis(ticker)

[*********************100%%**********************]  1 of 1 completed


Error in Criteria 2: 'Operating Cash Flow'


[*********************100%%**********************]  1 of 1 completed


Error in Criteria 2: 'Operating Cash Flow'


[*********************100%%**********************]  1 of 1 completed


...

Error in Criteria 2: 'Operating Cash Flow'


[*********************100%%**********************]  1 of 1 completed


Error in Criteria 1: 'Net Income'
Error in Criteria 2: 'Operating Cash Flow'
Error in Criteria 3: 'Total Assets'
Error in Criteria 4: 'Operating Revenue'
Error in Criteria 5: 'Long Term Debt And Capital Lease Obligation'
Error in Criteria 6: 'Total Liabilities Net Minority Interest'
Error in Criteria 7: 'Diluted Average Shares'
Error in Criteria 8: 'Total Revenue'
KeyError: 'Net Income'
KeyError: 'Stockholders Equity'


[*********************100%%**********************]  1 of 1 completed


Error in Criteria 1: 'Net Income'
Error in Criteria 2: 'Operating Cash Flow'
Error in Criteria 3: 'Total Assets'
Error in Criteria 4: 'Operating Revenue'
Error in Criteria 5: 'Long Term Debt And Capital Lease Obligation'
Error in Criteria 6: 'Total Liabilities Net Minority Interest'
Error in Criteria 7: 'Diluted Average Shares'
Error in Criteria 8: 'Total Revenue'
KeyError: 'Net Income'
KeyError: 'Stockholders Equity'


In [63]:
stock_analysis

{'AALI.JK': {'Average Daily Returns': -2.992250227229979e-05,
  'Beta Alpha': {'beta': array([0.035]), 'alpha': array([-0.])},
  'Sharpe Ratio': -0.03,
  'Piotroski F-Score': {'net_income': 833090000000.0,
   'roa': 0.0299875120724492,
   'op_revenue_v_net_income_margin': 17973953000000.0,
   'current_ratio': 0.30716554640793275},
  'Revenue Growth': -22.67492030276398,
  'Earning per Share': 432.8437198839821,
  'Price to Earning': 15.47902786205572,
  'Debt to Equity': 0.2135743776992211,
  'Free Cash Flow': '',
  'Return on Equity': 4.442555984422978,
  'Dividend Yield': 133.69811938805972},
 'ABBA.JK': {'Average Daily Returns': 0.001393156250978238,
  'Beta Alpha': {'beta': array([-0.047]), 'alpha': array([0.001])},
  'Sharpe Ratio': 0.45,
  'Piotroski F-Score': {'op_revenue_v_net_income_margin': 203504123516.0},
  'Revenue Growth': -6.034068612093069,
  'Earning per Share': -11.406882837566759,
  'Price to Earning': -4.470984819098833,
  'Debt to Equity': -20.456808251051843,
  'F

## Create Pandas DataFrame from the Calculation Result

In [64]:
analysis = pd.DataFrame.from_dict(stock_analysis, orient='index')
analysis.head()

,Average Daily Returns,Beta Alpha,Sharpe Ratio,Piotroski F-Score,Revenue Growth,Earning per Share,Price to Earning,Debt to Equity,Free Cash Flow,Return on Equity,Dividend Yield
AALI.JK,-0.000030,"{'beta': [0.035], 'alpha': [-0.0]}",-0.03,"{'net_income': 833090000000.0, 'roa': 0.029987...",-22.67492,432.84372,15.479028,0.213574,,4.442556,133.698119
ABBA.JK,0.001393,"{'beta': [-0.047], 'alpha': [0.001]}",0.45,{'op_revenue_v_net_income_margin': 20350412351...,-6.034069,-11.406883,-4.470985,-20.456808,,458.951936,0.0
ABDA.JK,0.000081,"{'beta': [-0.006], 'alpha': [0.0]}",0.07,"{'net_income': 138203753000.0, 'roa': 0.055777...",15.041351,3906.974495,1.279763,0.000000,,9.963071,
ABMM.JK,0.001542,"{'beta': [0.138], 'alpha': [0.001]}",0.73,{'op_revenue_v_net_income_margin': 642066803.0...,-40.656829,-0.012952,-282578.907036,5.803908,,-19.281866,20.048411
ACES.JK,-0.000341,"{'beta': [0.039], 'alpha': [-0.0]}",-0.21,"{'net_income': 733195452445.0, 'roa': 0.101596...",13.28681,42.825836,19.497576,0.163593,,13.764517,27.131018


In [65]:
#Number of Columns

analysis.columns

Index(['Average Daily Returns', 'Beta Alpha', 'Sharpe Ratio',
       'Piotroski F-Score', 'Revenue Growth', 'Earning per Share',
       'Price to Earning', 'Debt to Equity', 'Free Cash Flow',
       'Return on Equity', 'Dividend Yield'],
      dtype='object')

In [66]:
#Number of Null Values

analysis.isna().sum()

Average Daily Returns    10
Beta Alpha                0
Sharpe Ratio             50
Piotroski F-Score         0
Revenue Growth            0
Earning per Share         0
Price to Earning          0
Debt to Equity            0
Free Cash Flow            1
Return on Equity          0
Dividend Yield            0
dtype: int64

In [ ]:
analysis[]

In [67]:
#Number of Empty Strings

empty_counts = analysis.apply(lambda col: col[col == ""].count())
print(empty_counts)

Average Daily Returns      0
Beta Alpha               235
Sharpe Ratio               0
Piotroski F-Score          0
Revenue Growth           330
Earning per Share        336
Price to Earning           0
Debt to Equity             0
Free Cash Flow           867
Return on Equity         320
Dividend Yield            95
dtype: int64


## Write DataFrame to CSV

In [68]:
csv_filename = "fundamental_analysis_version2.csv"
analysis.to_csv(csv_filename)